In [1]:
import os, re, string
from collections import defaultdict


def pulizia(s):
    s = s.replace("\n", "")
    s = s.replace("\’", " ")
    s = s.replace("a'", "à")
    s = s.replace("e'", "è")
    s = s.replace("i'", "ì")
    s = s.replace("o'", "ò")
    s = s.replace("u'", "ù")
    s = re.sub(r'(?<=[.,?!\'])(?=[^\s])', r' ', s)
    s = s.translate(str.maketrans('', '', string.punctuation))
    return s

def capitalizza(testo):
    testo = " ".join([s.replace("\n", "").capitalize() for s in re.split(r'( *[\.\?!][\'"\)\]]* *)', testo)])
    testo = testo.replace(" . ", ". ")
    testo = testo.replace(" ? ", "? ")
    testo = testo.replace(" ! ", "! ")
    return testo

d = {}
d2 = defaultdict(list)

for filename in os.listdir('./Corpus/'):
    if filename.endswith('.txt'):
        with open(os.path.join('./Corpus/', filename)) as f:
            
            content = f.read()
            if content.isupper():
                content = capitalizza(content)
            normalized = content.lower()
            normalized = pulizia(normalized)
            
            filename = filename.split(".")[0]
            year, pres = (filename.split("-"))
            year = int(year)
            
            
        d[year] = {"presidente":pres, "nb_token":len(content.split()), "discorso_norm":normalized, "discorso_orig":content}
        d2[pres].append(normalized)
        
d3 = {}
for pres,los in d2.items():
    d3[pres] = {"discorso_norm":" ".join(los)}
d2 = None

In [2]:
from collections import Counter
import nltk
from nltk.corpus import stopwords

ignored_words = nltk.corpus.stopwords.words('italian')


def tokenize_without_sw(diz, ignored_words=ignored_words):
    return [w for w in diz["discorso_norm"].split() if w not in ignored_words]

for year, diz in d.items():
    tokens = tokenize_without_sw(diz)
    d[year]["piu_frequenti"] = {k:v for k,v in Counter(tokens).most_common(30)}

In [20]:
import collections, functools, operator

pri_rep = []
sec_rep = []

for year in d:
    if year<1994:
        pri_rep.append(d[year]["piu_frequenti"])
    else:
        sec_rep.append(d[year]["piu_frequenti"])

priRep = dict(functools.reduce(operator.add, map(collections.Counter, pri_rep)))
secRep = dict(functools.reduce(operator.add, map(collections.Counter, sec_rep)))


respub = []
for i in priRep:respub = []
for i in priRep:
    if i in secRep:
        respub.append([i, priRep[i], secRep[i]])
    else:
        respub.append([i, priRep[i], 1])
    if i in secRep:
        respub.append([i, priRep[i], secRep[i]])
    else:
        respub.append([i, priRep[i], 1])

for i in secRep:
    if i not in priRep:
        respub.append([i, 1, secRep[i]])


In [23]:
import csv

header = ['token', "freq. 1° Repubblica", "freq. 2° Repubblica"]

with open('respublica.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)

    # write the header
    writer.writerow(header)

    # write multiple rows
    writer.writerows(respub)

In [79]:
import nltk
from nltk.corpus import stopwords
from nltk.metrics.association import BigramAssocMeasures
from nltk.collocations import BigramCollocationFinder
from nltk.tokenize import RegexpTokenizer

def multiwords(tokenized_text):      
    bigram_measures = nltk.collocations.BigramAssocMeasures()                       
        ## oggetto: misure di associazione per bigrammi     
    finder2 = BigramCollocationFinder.from_words(tokenized_text, window_size=2)     
        ## oggetto: cercare collocazioni in finestre di taglia due     
    #ignored_words = nltk.corpus.stopwords.words('italian')                          
        ## (se aumenti la finestra cerchi degli skipgrams)
    #finder2.apply_word_filter(lambda w:w.lower() in ignored_words)                  
        ## applico filtro per collocazioni con stopwords (lower and uppercase)     
    scored2 = finder2.score_ngrams(bigram_measures.student_t)                       
        ## crea una lista di collocazioni in ordine di score di associazione 
        ##(raw_freq, student_t, likelihood_ratio, poisson_stirling...)     
    #return [(indice, collocazione) for indice, collocazione in enumerate(scored2)]
    return {ix:str(v) for ix,v in enumerate(scored2[:30])}

#multiwords(tokens)

In [80]:
from nltk.stem import SnowballStemmer

stemmer_snowball = SnowballStemmer('italian')    
    
for pres, diz in d3.items():
    tokens = tokenize_without_sw(diz)
    tokens_stem = [stemmer_snowball.stem(w) for w in tokens]
    d3[pres]["piu_frequenti_stem"] = {k:v for k,v in Counter(tokens_stem).most_common(30)}
    d3[pres]["30collocazioni"] = multiwords(tokens)

In [81]:
import spacy
nlp = spacy.load('it_core_news_sm')

def gulp(n_chars: int, n_words: int, n_sents: int) -> float:
    return (300 * n_sents / n_words) - (10 * n_chars / n_words) + 89

for year, diz in d.items():
    
    n_chars = len(diz["discorso_orig"])
    speech = nlp(diz['discorso_orig'])
    n_words = len(speech)
    n_sents = 0
    for sent in speech.sents:
        n_sents += 1
    
    tmp = []
    
    for token in speech:
        tmp.append(token.pos_)
    
    counter = dict(Counter(tmp)).items()
    len_per = {k:(v/d[year]["nb_token"])*100 for k,v in counter}
    d[year]["PoS_percentage"] = {k:v for k,v in sorted(len_per.items(),key=lambda i:i[1], reverse=True)}
    d[year]["indice_gulpease"] = gulp(n_chars, n_words, n_sents)

In [82]:
import json

with open('corpus_xdata.json', 'w') as f:
    json.dump(d, f)
    
with open('corpus_xpresidente.json', 'w') as f:
    json.dump(d3, f)

In [83]:
# todos

# bugfixing generico sulla normalizzazione

## lower / capitalize speeches in capslock
## trattini e punteggiatura speciale: verificare#
## rapp lunghezza, punteggiatura